# New Section

Here's the code to upload files from your local computer to your Colab environment. After running the cell, you will see a button to select and upload files.

In [3]:
import os
import pandas as pd
import re
from google.colab import files

# --- Step 1: File Upload and Sheet Selection ---

# Get the list of all files in the current directory (optional, for display)
files_in_directory = os.listdir()
print("Files in the current directory before potential deletion:")
for filename in files_in_directory:
    print(filename)

# Code to delete specific files if needed (uncommented to clear previous upload)
files_to_delete = ['AF.xlsx'] # Example: list files to delete if needed. Update this list if needed.
for filename in files_to_delete: # Changed to iterate over a specific list if needed
    if filename in files_in_directory:
        try:
            os.remove(filename)
            print(f"Successfully removed '{filename}'.")
        except Exception as e:
            print(f"Error removing file '{filename}': {e}")

# print(f"\nAttempted to delete {len(files_in_directory)} files. Successfully deleted {files_deleted_count} files.") # This line might be misleading if not deleting all files

# List files again to confirm deletion (optional)
files_in_directory_after = os.listdir()
print("\nFiles in the current directory after potential deletion:")
if files_in_directory_after:
    for filename in files_in_directory_after:
        print(filename)
else:
    print("Directory is empty.")


print("\nUploading files...")
uploaded = files.upload()

for filename in uploaded.keys():
  print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')


# Assuming the uploaded file is an Excel file and its name is the first uploaded file
# If you uploaded multiple files or the name is different, adjust this.
selected_sheets_to_process = [] # Initialize list to store selected sheet names
excel_file_path = None # Initialize excel_file_path

if uploaded:
    excel_file_path = list(uploaded.keys())[0] # Get the name of the first uploaded file
    print(f"\nProcessing uploaded file: {excel_file_path}")

    try:
        # Read the Excel file to get the sheet names
        excel_file = pd.ExcelFile(excel_file_path)
        excel_sheets = excel_file.sheet_names

        print(f"\nSheet names in '{excel_file_path}':")
        for i, sheet_name in enumerate(excel_sheets):
            print(f"{i+1}. {sheet_name}")

        # Prompt the user to select multiple sheets
        while True:
            choice_str = input("\nEnter the numbers or names of the sheets you want to process, separated by commas (e.g., 1,3 or Sheet1,Sheet3): ")
            choices = [c.strip() for c in choice_str.split(',')] # Split by comma and remove whitespace
            selected_sheets_to_process = []
            invalid_choices = []

            for choice in choices:
                if choice.isdigit():
                    choice_index = int(choice) - 1
                    if 0 <= choice_index < len(excel_sheets):
                        selected_sheets_to_process.append(excel_sheets[choice_index])
                    else:
                        invalid_choices.append(choice)
                elif choice in excel_sheets:
                    selected_sheets_to_process.append(choice)
                else:
                    invalid_choices.append(choice)

            if not selected_sheets_to_process:
                print("No valid sheets selected. Please try again.")
            elif invalid_choices:
                print(f"Invalid choices: {', '.join(invalid_choices)}. Please enter valid sheet numbers or names.")
            else:
                print(f"You selected sheets: {', '.join(selected_sheets_to_process)}")
                break # Exit the loop if all selections are valid

        # The list selected_sheets_to_process now contains the names of the chosen sheets.
        # Proceed to the next step (Special Character Check and Report Generation)

    except FileNotFoundError:
        print(f"Error: The file '{excel_file_path}' was not found. Please make sure the file is uploaded and the name is correct.")
        excel_file_path = None # Reset if file not found to prevent subsequent errors
    except Exception as e:
        print(f"An error occurred during file processing or sheet selection: {e}")
        excel_file_path = None # Reset on error


# --- Step 2: Special Character Check and Report Generation ---

if excel_file_path and selected_sheets_to_process: # Proceed only if file was uploaded and sheets were selected
    try:
        excel_file = pd.ExcelFile(excel_file_path)

        # List to store the findings for the report
        qc_report_data = []

        # Prompt the user to enter a column to bypass (optional)
        column_to_bypass = input("\nEnter the name of a column to bypass special character check (leave blank to check all columns): ").strip()
        if column_to_bypass:
            print(f"\nBypassing special character check for column: '{column_to_bypass}'")
        else:
            print("\nChecking all columns for special characters.")

        # Iterate through each selected sheet
        for sheet_name in selected_sheets_to_process:
            print(f"\n--- Checking special characters in sheet: '{sheet_name}' ---")

            # Load the current sheet into a temporary DataFrame
            try:
                current_sheet_df = excel_file.parse(sheet_name)

                # Iterate through each column in the current sheet's DataFrame
                for column in current_sheet_df.columns:
                     # Check if the current column should be bypassed (case-insensitive check added)
                    if column_to_bypass and column.strip().lower() == column_to_bypass.lower():
                        print(f"\n  Bypassing column: '{column}' in sheet '{sheet_name}' as requested.")
                        continue # Skip to the next column

                    # print(f"\n  Checking column: '{column}' in sheet '{sheet_name}'") # Optional: uncomment for detailed progress

                    # Check each value in the column
                    for index, value in enumerate(current_sheet_df[column]): # Added index to potentially locate the row
                        # Convert value to string to handle different data types, and handle potential NaN values
                        value_str = str(value) if pd.notna(value) else ""
                        # Find all characters that are in the user-provided list, common special characters,
                        # OR a space, while excluding alphanumeric and underscore.
                        # Modified regex to explicitly include space ' '
                        found_chars = re.findall(r'[ ~`!@#$%^&*()_\-+=|\]}[{\'";:/?.>,<\\]', value_str)

                        if found_chars:
                            # Add findings to the report data list
                            for char in found_chars:
                                qc_report_data.append({
                                    'File Name': os.path.basename(excel_file_path),
                                    'Sheet Name': sheet_name,
                                    'Column Name': column,
                                    'Special Character Found': char,
                                    'Cell Name': f'{column}, Row {index + 2}', # Changed to show column name and row index
                                })

                # print(f"  Finished checking column: '{column}' in sheet '{sheet_name}'") # Optional: uncomment for detailed progress


            except Exception as e:
                print(f"  An error occurred while processing sheet '{sheet_name}': {e}")

        # After checking all selected sheets, create the report DataFrame
        if qc_report_data:
            qc_report_df = pd.DataFrame(qc_report_data)

            # Define the report file name
            report_file_name = "ValidationErrorReport.xlsx" # Exporting as XLSX

            # Get the directory of the input file (assuming it's the current directory after upload)
            input_file_directory = os.path.dirname(excel_file_path) if excel_file_path else "."

            # Define the full path for the report file
            report_file_path = os.path.join(input_file_directory, report_file_name)

            # Export the report DataFrame to an Excel file
            try:
                qc_report_df.to_excel(report_file_path, index=False)
                print(f"\n--- QC Report Generated ---")
                print(f"Validation Error Report exported successfully to: {report_file_path}")
                # Display the report DataFrame
                print("\nFirst 5 rows of the QC Report:")
                display(qc_report_df.head())

                # --- Step 3: Download the Report ---
                print("\nAttempting to download the report...")
                files.download(report_file_path) # Download the generated report


            except Exception as e:
                print(f"\nError exporting or downloading the QC Report: {e}")

        else:
            print("\nNo special characters found in the selected sheets. No report generated.")


    except FileNotFoundError:
        # This block might not be necessary if the file existence is checked in Step 1
        print(f"Error: The Excel file '{excel_file_path}' was not found during processing.")
    except Exception as e:
        print(f"An error occurred during special character check or report generation: {e}")

else:
    if not uploaded:
        print("\nNo file was uploaded. Please upload a file to proceed.")
    elif not selected_sheets_to_process:
         print("\nNo sheets were selected. Please select sheets to proceed with the special character check.")

Files in the current directory before potential deletion:
.config
AF.xlsx
ValidationErrorReport.xlsx
sample_data
Successfully removed 'AF.xlsx'.

Files in the current directory after potential deletion:
.config
ValidationErrorReport.xlsx
sample_data

Uploading files...


Saving AF.xlsx to AF.xlsx
User uploaded file "AF.xlsx" with length 34705 bytes

Processing uploaded file: AF.xlsx

Sheet names in 'AF.xlsx':
1. Sheet 1
2. Sheet 2
3. Sheet 3

Enter the numbers or names of the sheets you want to process, separated by commas (e.g., 1,3 or Sheet1,Sheet3): 1,2,3
You selected sheets: Sheet 1, Sheet 2, Sheet 3

Enter the name of a column to bypass special character check (leave blank to check all columns): Period

Bypassing special character check for column: 'Period'

--- Checking special characters in sheet: 'Sheet 1' ---

  Bypassing column: 'Period' in sheet 'Sheet 1' as requested.

--- Checking special characters in sheet: 'Sheet 2' ---

  Bypassing column: 'Period' in sheet 'Sheet 2' as requested.

--- Checking special characters in sheet: 'Sheet 3' ---

  Bypassing column: 'Period' in sheet 'Sheet 3' as requested.

--- QC Report Generated ---
Validation Error Report exported successfully to: ValidationErrorReport.xlsx

First 5 rows of the QC Report:


,File Name,Sheet Name,Column Name,Special Character Found,Cell Name
0,AF.xlsx,Sheet 1,Total_Sell_Out_Units_G_Unit,.,"Total_Sell_Out_Units_G_Unit, Row 2"
1,AF.xlsx,Sheet 1,Total_Sell_Out_Units_G_Unit,.,"Total_Sell_Out_Units_G_Unit, Row 3"
2,AF.xlsx,Sheet 1,Total_Sell_Out_Units_G_Unit,.,"Total_Sell_Out_Units_G_Unit, Row 4"
3,AF.xlsx,Sheet 1,Total_Sell_Out_Units_G_Unit,.,"Total_Sell_Out_Units_G_Unit, Row 5"
4,AF.xlsx,Sheet 1,Total_Sell_Out_Units_G_Unit,.,"Total_Sell_Out_Units_G_Unit, Row 6"



Attempting to download the report...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>